In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import math
from tqdm.notebook import tqdm
from typing import Iterable, List
from timeit import default_timer as timer
import torch
import torch.nn as nn
import pandas as pd
from torch import Tensor
from torch.nn import Transformer

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import multi30k, Multi30k

In [3]:
use_cuda = torch.cuda.is_available()
DEVICE = torch.device("cuda:0" if use_cuda else "cpu")
print("Available device: {}".format(DEVICE))

Available device: cuda:0


In [4]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download zh_core_web_sm
!pip install -U torchdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 33.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)


In [5]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00


In [6]:
!pip install portalocker

# Load Datasets

In [7]:
from datasets import load_dataset
train_dataset = load_dataset(
                  "ngxingyu/iwslt17_google_trans_scores_sentiments",
                  # streaming=True,
                  split="train",
                )
test_dataset = load_dataset(
                  "ngxingyu/iwslt17_google_trans_scores_sentiments",
                  # streaming=True,
                  split="test",
                )
validation_dataset = load_dataset(
                    "ngxingyu/iwslt17_google_trans_scores_sentiments",
                    # streaming=True,
                    split="validation",
                )
print(train_dataset[0])
print(test_dataset[0])
print(validation_dataset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/229736 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/875 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8549 [00:00<?, ? examples/s]

{'bleurt_score': 0.5358738303184509, 'comet_score': 0.8854652047157288, 'en': "Thank you so much, Chris. And it's truly a great honor to have the opportunity to come to this stage twice; I'm extremely grateful.", 'google_zh': '非常感谢你，克里斯。非常荣幸有机会两次登上这个舞台；我非常感激。', 'zh': '非常谢谢，克里斯。的确非常荣幸 能有第二次站在这个台上的机会，我真是非常感激。', 'en_sentiment': [-1.896572470664978, -1.776237964630127, 3.2432024478912354], 'zh_sentiment': [-1.788645625114441, -1.4522335529327393, 2.8387067317962646]}
{'bleurt_score': 0.6811590194702148, 'comet_score': 0.9185108542442322, 'en': 'Several years ago here at TED, Peter Skillman  introduced a design challenge  called the marshmallow challenge.', 'google_zh': '几年前，Peter Skillman 在 TED 上提出了一项名为“棉花糖挑战”的设计挑战。', 'zh': '几年前，在TED大会上， Peter Skillman 介绍了一个设计挑战 叫做“棉花糖挑战”', 'en_sentiment': [-3.196884870529175, 1.5497978925704956, 1.2930272817611694], 'zh_sentiment': [-2.8004977703094482, 2.4916672706604004, -0.260890930891037]}
{'bleurt_score': 0.44820863008499146, 'comet_score': 0.8485136

In [8]:
print(train_dataset, validation_dataset)

Dataset({
    features: ['bleurt_score', 'comet_score', 'en', 'google_zh', 'zh', 'en_sentiment', 'zh_sentiment'],
    num_rows: 229736
}) Dataset({
    features: ['bleurt_score', 'comet_score', 'en', 'google_zh', 'zh', 'en_sentiment', 'zh_sentiment'],
    num_rows: 875
})


In [9]:
SRC_LANGUAGE = 'en'
TGT_LANGUAGE = 'zh'

# Placeholders
token_transform = {}
vocab_transform = {}

In [10]:
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='zh_core_web_sm')

# Helper function to yield list of tokens
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
    language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}
    for data_sample in data_iter:
        print('here', data_sample[language_index[language]])
        yield token_transform[language](data_sample[language_index[language]])

In [11]:
from torch.utils.data import Dataset
from typing import Iterable, List

class CustomDataset(Dataset):
    def __init__(self, data_iter: Iterable, token_transform):
        self.data_iter = data_iter
        self.token_transform = token_transform
        self.process_data_iter()

    def __len__(self):
        return len(self.data_iter)

    def __getitem__(self, idx):
        sample = self.data_iter[idx]
        return sample

    def process_data_iter(self):
        en_data = self.data_iter['en']
        zh_data = self.data_iter['zh']
        iter = []
        for i in range(len(self.data_iter)):
          data_sample = {}
          data_sample['en'] = en_data[i]
          data_sample['zh'] = zh_data[i]
          iter.append(data_sample)
        self.data_iter = iter

    def yield_tokens(self, language: str):
        with tqdm(total=len(self.data_iter)) as progress_bar:
            for data_sample in self.data_iter:
                yield self.token_transform[language](data_sample[language])
                progress_bar.update(1)

In [12]:
from torchtext.datasets import multi30k, Multi30k
custom_train_dataset = CustomDataset(train_dataset, token_transform)
custom_validation_dataset = CustomDataset(validation_dataset, token_transform)
custom_test_dataset = CustomDataset(test_dataset, token_transform)

# Define special symbols and indices
PAD_IDX, UNK_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
# Make sure the tokens are in order of their indices to properly insert them in vocab
special_symbols = ['<PAD>', '<UNK>', '<SOS>', '<EOS>']

for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(custom_train_dataset.yield_tokens(ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set `UNK_IDX` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

  0%|          | 0/229736 [00:00<?, ?it/s]

  0%|          | 0/229736 [00:00<?, ?it/s]

In [13]:
print('EN vocab length:', len(vocab_transform[SRC_LANGUAGE]))
print('ZH vocab length:', len(vocab_transform[TGT_LANGUAGE]))

EN vocab length: 62604
ZH vocab length: 101152


## Define Positional Encodings

### Absolute Positional Encoding

In [14]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

### Relative Positional Encoding

In [15]:
''' Adapted from Gazelle's repository: https://github.com/gazelle93/Transformer-Various-Positional-Encoding/blob/main/positional_encoders.py '''
class RelativePositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(RelativePositionalEncoding, self).__init__()
        self.maxlen = maxlen
        self.embeddings_table = nn.Parameter(torch.Tensor(maxlen * 2 + 1, emb_size))
        nn.init.xavier_uniform_(self.embeddings_table)

    def forward(self, seq_len_q, seq_len_k):
        range_vec_q = torch.arange(seq_len_q)
        range_vec_k = torch.arange(seq_len_k)
        relative_matrix = range_vec_k[None, :] - range_vec_q[:, None]
        clipped_relative_matrix = torch.clamp(relative_matrix, -self.maxlen, self.maxlen)
        relative_position_matrix = clipped_relative_matrix + self.maxlen
        embeddings = self.embeddings_table[relative_position_matrix]
        return embeddings

In [16]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Construct Model

In [17]:
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.relative_positional_encoding = RelativePositionalEncoding(emb_size, dropout=dropout) # Replace Positional Encoding Layer with the relative positional encoding

    def forward(self,
              src: Tensor,
              trg: Tensor,
              src_mask: Tensor,
              tgt_mask: Tensor,
              src_padding_mask: Tensor,
              tgt_padding_mask: Tensor,
              memory_key_padding_mask: Tensor):


        # Apply paddings for matrix manipulation
        src_positional_encoding = self.relative_positional_encoding(seq_len_q=src.shape[1], seq_len_k=src.shape[1])
        src_padded_tensor = torch.nn.functional.pad(torch.zeros_like(self.src_tok_emb(src)), (0, 0, 0, 0, 0, src_positional_encoding.shape[0] - src.shape[1]))

        tgt_positional_encoding = self.relative_positional_encoding(seq_len_q=trg.shape[1], seq_len_k=trg.shape[1])
        tgt_padded_tensor = torch.nn.functional.pad(torch.zeros_like(self.tgt_tok_emb(trg)), (0, 0, 0, 0, 0, tgt_positional_encoding.shape[0] - trg.shape[1]))

        src_emb = self.src_tok_emb(src) + src_padded_tensor
        tgt_emb = self.tgt_tok_emb(trg) + tgt_padded_tensor


        # Pass embeddings through the transformer
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None, src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)


    def encode(self, src: Tensor, src_mask: Tensor):
        src_positional_encoding = self.relative_positional_encoding(seq_len_q=src.shape[1], seq_len_k=src.shape[1])
        src_pad_amounts = [max(0, src.shape[i] - src_positional_encoding.shape[i]) for i in range(len(src.shape))]
        src_padded_tensor = torch.nn.functional.pad(src_positional_encoding, (0, 0, 0, 0, 0, src_pad_amounts[0]))
        return self.transformer.encoder(self.src_tok_emb(src) +  src_padded_tensor, src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        tgt_positional_encoding = self.relative_positional_encoding(seq_len_q=tgt.shape[1], seq_len_k=tgt.shape[1])
        tgt_pad_amounts = [max(0, tgt.shape[i] - tgt_positional_encoding.shape[i]) for i in range(len(tgt.shape))]
        tgt_padded_tensor = torch.nn.functional.pad(tgt_positional_encoding, (0, 0, 0, 0, 0, tgt_pad_amounts[0]))
        return self.transformer.decoder(self.tgt_tok_emb(tgt) + tgt_padded_tensor, memory, tgt_mask)

In [18]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask.bool()


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [19]:
# Helper method to club together sequential operations
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# method to add SOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([SOS_IDX]), torch.tensor(token_ids), torch.tensor([EOS_IDX])))


# `src` and `tgt` language text transforms to convert raw strings into tensors indices
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], # Tokenization
                                               vocab_transform[ln], # Vectorization
                                               tensor_transform)    # Add SOS/EOS and create tensor


# Method to collate data samples into batch tensors
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for i in range(len(batch)):
        src_sample, tgt_sample = batch[i][SRC_LANGUAGE], batch[i][TGT_LANGUAGE]
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))
    # for src_sample, tgt_sample in batch:
    #     print(batch[0])
    #     print(text_transform[SRC_LANGUAGE](batch[0]['en'].rstrip("\n")))
    #     src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
    #     tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))
    # pad_sequence is an in-built method proivded by torch package
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

# Train model

In [20]:
def train_epoch(model, optimizer, criterion):
    model.train()
    losses = 0

    train_dataloader = DataLoader(custom_train_dataset.data_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in tqdm(train_dataloader, total=len(list(train_dataloader))):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        # Remove last entry an all target sequences (typically PAD, can be EOS)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        optimizer.zero_grad()

        # Remove <SOS> from all targets
        tgt_out = tgt[1:, :]

        loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()

        optimizer.step()
        losses += loss.item()

    return losses / len(list(train_dataloader))


def evaluate(model):
    model.eval()
    losses = 0

    val_dataloader = DataLoader(custom_validation_dataset.data_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

    for src, tgt in tqdm(val_dataloader, total=len(list(val_dataloader))):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)

        tgt_input = tgt[:-1, :]

        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)

        tgt_out = tgt[1:, :]

        loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(val_dataloader))

In [21]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 8
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

# Create model
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

# Initialize weights
for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

# Move model to device (ideally GPU, otherwise CPU)
transformer = transformer.to(DEVICE)

# Define loss function
criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Define optimizer
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [22]:
NUM_EPOCHS = 2
best_val_loss = 1e5


for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train_epoch(transformer, optimizer, criterion)
    end_time = timer()
    val_loss = evaluate(transformer)
    if best_val_loss > val_loss:
      best_val_loss = val_loss
      # torch.save(transformer.state_dict(), '/content/drive/My Drive/project/transformer/transformer_relational_positional_encoding_model_2_epochs.pt' ) # uncomment to save model

    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, "f"Epoch time (total) = {(end_time - start_time):.3f}s"))


  0%|          | 0/28717 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 1, Train loss: 5.466, Val loss: 5.627, Epoch time (total) = 3515.102s


  0%|          | 0/28717 [00:00<?, ?it/s]

  0%|          | 0/110 [00:00<?, ?it/s]

Epoch: 2, Train loss: 4.866, Val loss: 5.414, Epoch time (total) = 3534.754s


In [23]:
# Method to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

def beam_search_decode(model, src, src_mask, max_len, start_symbol, beam_size):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)

    # Initialize the beam search candidates with the start symbol
    beams = [([start_symbol], 0)]

    for _ in range(max_len - 1):
        new_beams = []

        for ys, score in beams:
            if ys[-1] == EOS_IDX:
                new_beams.append((ys, score))
                continue

            # Convert the current token sequence to tensor
            ys_tensor = torch.tensor(ys, dtype=torch.long, device=DEVICE).unsqueeze(0)

            # Generate mask for the target sequence
            tgt_mask = (generate_square_subsequent_mask(ys_tensor.size(1)).type(torch.bool)).to(DEVICE)

            # Decode the next token probabilities
            out = model.decode(ys_tensor, memory, tgt_mask)
            out = out.transpose(0, 1)

            # Get the probabilities for the next tokens
            prob = model.generator(out[:, -1])

            # Get top k candidates and their indices
            topk_probs, topk_indices = torch.topk(prob, beam_size)

            # Expand the current candidates with the topk candidates
            for i in range(beam_size):
                new_score = score + topk_probs[0][i].item()
                new_candidate = ys + [topk_indices[0][i].item()]
                new_beams.append((new_candidate, new_score))

        # Sort the new beams based on their scores
        new_beams.sort(key=lambda x: x[1], reverse=True)

        # Keep only the top k beams
        beams = new_beams[:beam_size]

    return beams


# Actual method to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(model, src, src_mask, max_len=num_tokens+5, start_symbol=SOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<SOS>", "").replace("<EOS>", "")

def translate_dataset(model, validation_dataset):
    translations = []
    for sample in tqdm(validation_dataset, total=len(validation_dataset)):
        src_sentence = sample[SRC_LANGUAGE]
        tgt_sentence = sample[TGT_LANGUAGE]
        # Translate the source sentence using the model
        translated_sentence = translate(model, src_sentence)
        # Append the translation along with source and target sentences
        translations.append({'en': src_sentence, 'zh': tgt_sentence, 'translated_output': translated_sentence})

    return pd.DataFrame(translations)

In [24]:
print(translate(transformer, "Several years ago here at TED, Peter Skillman introduced a design challenge called the marshmallow challenge."))
print(translate(transformer, "And it was a huge success."))
print(translate(transformer, "That number's rising, and it's rising really rapidly, and it's disgraceful."))
print(translate(transformer, "And the idea's pretty simple: Teams of four have to build the tallest free-standing structure out of 20 sticks of spaghetti, one yard of tape, one yard of string and a marshmallow."))

 几 年前 ， 一个 挑战 叫做 TED 的 挑战 。 
 它 是 一个 巨大 的 成功 成功 的 成功 
 这是 个 非常 快 的 数字 ， 而且 它 非常 快 ， 而且 它 在 下降 。 
 这个 想法 很 简单 ： 从 一个 到 20 个 到 20 个 不同 的 结构 中 ， 一个 简单 的 想法 ， 一个 简单 的 结构 ， 一个 在 一个 小 的 时候 ， 一个 能 建立 一个 很 大 的 东西 ， 然后


In [25]:
translations = translate_dataset(transformer, custom_test_dataset.data_iter)

  0%|          | 0/8549 [00:00<?, ?it/s]

In [ ]:
def clean_tranlsations(text):
    # Replace underscores with spaces
    cleaned_text = text.replace("▁", " ")
    # For Chinese text, trim leading and trailing spaces, but preserve spaces between characters
    if text.strip() == text:  # Check if the text has no leading or trailing spaces
        cleaned_text = cleaned_text.strip()  # Trim leading and trailing spaces
    return cleaned_text

translations['translated_output'] = translations['translated_output'].apply(clean_tranlsations)
translations.to_csv('/content/drive/My Drive/project/transformer/translations_relational.csv', index=True)